In [ ]:
# cos(z) + ln(c)

%matplotlib tk
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
from matplotlib import cm


width, height = 1000, 1000 # defines quality of meshgrid
max_iter = 300 #how many it takes before program decides if it is bounded
zoom_factor = 0.05  # decrease number to zoom in, increase to zoom out

# Initial bounds
real_min, real_max = -25, 25 # fixed by the real axis
imag_min, imag_max = -25, 25 # fixed by the imaginary axis


def compute_fractal(real_min, real_max, imag_min, imag_max): # this part actually computes & processes the fractal before plotting it
    real = np.linspace(real_min, real_max, width) # real axis 
    imag = np.linspace(imag_min, imag_max, height) # imaginary axis
    real_grid, imag_grid = np.meshgrid(real, imag) # creates meshgrid out of real and imaginary axis—— the complex plane!
    c = real_grid + 1j * imag_grid 
    #This one's a little complicated. 
    #c is a complex number that is fixed throughout a single iteration, a.k.a it CAN and WILL change to compute the entire image,
    #just not within a single instance. In this case, c is being defined as a number that's on the real and imaginary axis. 
    #i.e, if c = 1, it really can also mean c = 1 + 0i, or 'j' in this case. That being said, c can be 0 + 2j, or 30.2 + 2.5j——
    #it really depends on what you're setting c to, and where z goes. Fun!!
    
    z = np.zeros_like(c)
    #Copies the array format of c, since z can also be a complex number too. z is always changing in every single iteration, 
    #it allows us to see whether the series diverges or orbits indefinitely between a sequence of numbers—— if it is bounded. 
    
    divergence_iter = np.zeros(c.shape, dtype=int) # tracks how many iterations it takes for the series to diverge
    mask = np.ones(c.shape, dtype=bool) 
    # this basically lets all the points in the meshgrid start off on a clean slate, 
    # and if it diverges it lets the computer know it can stop computing it so that the 
    # program runs faster :)

    for i in range(max_iter):
        with np.errstate(divide='ignore', invalid='ignore', over='ignore'): # safeguard for errors so it doesnt crash lol
            z[mask] = np.cos(z[mask]) + np.log(c[mask]) # actual equation: z = cos(z) + ln(c)
            mask_new = np.abs(z) <= 10 # escape test
            divergence_iter[mask & ~mask_new] = i # keeps track of how many iterations it takes for series to diverge.
            mask &= mask_new # updates which ones have diverged and which have not (bounded).

    divergence_iter[mask] = max_iter 
    # clean up of which numbers did not diverge past the max_iterations and therefore belongs to the fractal set.
    return divergence_iter


def plot_fractal(ax, divergence_iter, real_min, real_max, imag_min, imag_max): #plotting time!!
    ax.clear() #now its time to actually plot these points on a graph to create fractal images.
    ax.imshow(
        divergence_iter,
        cmap=cm.cubehelix,
        extent=(real_min, real_max, imag_min, imag_max),
        origin='lower'
    )
    ax.set_title("z = cos(z) + ln(c)") #equation so you know, + real and imag axis
    ax.set_xlabel("real")
    ax.set_ylabel("imaginary")
    plt.draw()

# Click event callback
def on_click(event):
    global real_min, real_max, imag_min, imag_max

    if event.inaxes:
        
        x_center, y_center = event.xdata, event.ydata #this tracks the data of the mouse (and where it wanders)

        
        real_range = (real_max - real_min) * zoom_factor #this computes the new bounds by multiplying by the zoom.
        imag_range = (imag_max - imag_min) * zoom_factor
        real_min = x_center - real_range / 2
        real_max = x_center + real_range / 2
        imag_min = y_center - imag_range / 2
        imag_max = y_center + imag_range / 2

        
        divergence_iter = compute_fractal(real_min, real_max, imag_min, imag_max) #this recomputes the entire plot to the 'zoomed' in intervals
        plot_fractal(ax, divergence_iter, real_min, real_max, imag_min, imag_max) # this plots it


fig, ax = plt.subplots(figsize=(10, 10)) # first render!
divergence_iter = compute_fractal(real_min, real_max, imag_min, imag_max)
plot_fractal(ax, divergence_iter, real_min, real_max, imag_min, imag_max)


fig.canvas.mpl_connect('button_press_event', on_click) #now connect the two together!!
plt.tight_layout()
plt.show()
